In [2]:
import torch
import torchvision
import torchvision.transforms as tr
#데이터 불러오며 전처리를 할 수 있게
from torch.utils.data import DataLoader, Dataset
#batchsize 형태로 변환가능 / Dataset은 튜닝
import numpy as np

#1. 파이토치 제공 데이터 사용

In [3]:
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])
#8x8로 변환후 tensor로 변환

# #Trnasforms on PIL Image
# pad, grayscale, randomcrop, normilize...
# transforms on torch.*Tensor - tensro importtorchvision.transforms.ToPILImage(mode=None)

#이 중 필요한 전처리 작업compose안에

In [4]:
trainset=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset=torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
trainset[0][0].size()
#tuple 형태로 저장, 이 경우는 3channel의 8x8 tensor

torch.Size([3, 8, 8])

In [6]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader (testset, batch_size=50, shuffle=True, num_workers=2)
#num_workers similar to n_jobs


In [7]:
len(trainloader)
#batchsize50개가 1000개있음을 의미

1000

In [8]:
#실제 값을 확인하고 싶으면
dataiter = iter(trainloader)
images, labels = dataiter.next()


In [9]:
images.size()
#batch_size is 50, 3channel, 8x8

torch.Size([50, 3, 8, 8])

##transforms --> torchvision.datasets --> DataLoader

#2. 같은 클래스 별 폴더 이미지 데이터 이용

In [ ]:
#ex) ./class/tiger  & ./class/lion

transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
#전처리 방법 정의
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
#class 폴더 안의 image들을 searching 후 각각 다른 폴더에 대해 label 및 전처리
trainloader = DataLoader(trainset, batch_size=1, shuffle=False, num_workers=2)
print(len(trainloader))

FileNotFoundError: ignored

In [ ]:
trainset[0][0].size()

torch.Size([3, 8, 8])

#3. 개인 데이터 사용 (2types)

In [ ]:
#import preprocessing

train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

#preprocessing
#train_images, train_labels = preprocessing(train_images, train_labels)

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [ ]:
#Dataset의 상속을 받을 class의 기본적인 형식
#

class TensorData(Dataset) :
  def __init__(self, x_data,y_data) :
    #외부 데이터를 쓸경우 이런식으로, 내부 데이터를 쓸경우는 self만
    self.x_data = torch.FloatTensor(x_data)
    #들어온 데이터를 tensor로 변환
    self.x_data = self.x_data.permute(0,3,1,2)    #이미지 개수, 채널 수, 이미지 너비 및 높이
    #즉, train_image.shape의 값은 (20,3,32,32)가 됨.
    # train_images의 순서인 batch_size, channel, w*h를 변경
    self.y_data = torch.LongTensor(y_data)
    #y또한 tensor로 변경 / torch.후에 long/float등으로 구체적으로 변경 가능
    self.len = self.y_data.shape[0]

  def __getitem__(self, index) :
    return self.x_data[index], self.y_data[index]
    #x 와 y를 tuple의 형태로 변환

  def __len__(self) :
    return self.len

In [ ]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [ ]:
train_data[0][0].size()

torch.Size([3, 32, 32])

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
images.size()

torch.Size([10, 3, 32, 32])

torchvision.datasets.ImageFolder를 안쓰는 이유?

1. 폴더 정리를 못할 경우
- 다른 작업과 공용으로 사용
- 혹은 SQL같은 곳에서 넘어오는 경우

2. pytorch에서 제공하는 transform의 종류가 제한적임
- 따라서 python의 preprocessing을 거친 후 Tensor로 추가 변환을 하는 것이 다양한 경우 가능함

#4. pytorch로만 전처리

In [ ]:
####기본 형태

#########from torch.utils.data import Dataset

# class MyDataset(Dataset) :
#   def __init__(self) : 데이터를 가져오고 전처리 방식을 결정
#   def __getitem__(self, index) : transform에 따라 형식을 np or tesor로 변환 후 데이터를 tuple로 묶음
#   def __len__(self) : 길이

#추가
#class ToTensor() : Tensor로 변환
#class LinearTensor() : 선형변환을 주려면 어떤 방식으로 변환을 할 것인가

In [ ]:
class MyDataset(Dataset) :

  def __init__(self, x_data, y_data, transform=None) :

    self.x_data = x_data
    self.y_data = y_data
    self.transform=transform
    self.len = len(y_data)

  def __getitem__ (self, index) :
    sample = self.x_data[index], self.y_data[index]
#tuple형태로 x 와 y를 변환
    if self.transform :
      sample = self.transform(sample)
      #tuple값을 반환하기 전 transform진행
    return sample
    #transform이 None이면 numpy로 반환, 만약 한다면 tensor로 반환
  
  def __len__(self) :
    return self.len


class ToTensor :
  def __call__(self, sample) :
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    return inputs, torch.LongTensor(labels)
    #나가는 값이 tensor


class LinearTensor :
  def __init__(self, slope=1, bias=0) :
    self.slope = slope
    self.bias = bias

  def __call__ (self, sample) :
    inputs, labels = sample
    inputs = self.slope*inputs + self.bias
    return inputs, labels

In [ ]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)])
#linear는 2x+5의 형식
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

In [ ]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
dataiter1 = iter(train_loader1)
images1,labels1=dataiter1.next()

In [ ]:
images1

tensor([[[[389.,  55., 493.,  ..., 111., 473., 441.],
          [159.,  67.,  71.,  ..., 327., 501., 485.],
          [  5., 273., 203.,  ..., 209., 397., 169.],
          ...,
          [375., 211., 431.,  ..., 419., 299., 313.],
          [451., 217., 423.,  ..., 161., 353., 175.],
          [471., 123., 403.,  ...,  25., 231., 129.]],

         [[ 25., 223., 319.,  ..., 489., 347.,  41.],
          [425.,  69., 501.,  ..., 431., 409., 339.],
          [501., 511., 457.,  ..., 189., 337., 489.],
          ...,
          [115., 221., 451.,  ..., 393., 373., 419.],
          [443., 231., 179.,  ..., 515., 455., 217.],
          [473., 473.,  31.,  ..., 339., 409., 121.]],

         [[507., 241., 149.,  ..., 261., 375., 255.],
          [ 31., 391., 259.,  ..., 179., 125., 345.],
          [221., 229., 495.,  ..., 195., 325., 483.],
          ...,
          [377., 157., 513.,  ...,   7.,  39.,  79.],
          [121., 485., 155.,  ..., 171., 365., 161.],
          [ 25.,  43., 505.,  ...

trans = tr.Compose([tr.ToTensor()])
이거 쓰면 안되냐?

- class를 사용하여 받은 데이터는 numpy임
- tr.ToTensor()로 받는 형식은 PIL Image를 바탕으로 사용하기 때문에 타입이 달라서 실행이 안됨

PIL을 사용하는 방법

In [ ]:
class MyDataset(Dataset) :

  def __init__(self, x_data, y_data, transform=None) :

    self.x_data = x_data
    self.y_data = y_data
    self.transform=transform
    self.len = len(y_data)

  def __getitem__ (self, index) :
    sample = self.x_data[index], self.y_data[index]
#tuple형태로 x 와 y를 변환
    if self.transform :
      sample = self.transform(sample)
      #tuple값을 반환하기 전 transform진행
    return sample
    #transform이 None이면 numpy로 반환, 만약 한다면 tensor로 반환
  
  def __len__(self) :
    return self.len

class MyTransform :
  def __init__(self,sample) :
    inputs = labels, sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    labels = torch.FloatTensor(labels)

    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    final_output = transf(inputs)

    return final_output, labels

In [ ]:
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True)

In [ ]:
first_data = ds2[0]
feature, labels = first_data
print(type(features), type(lables))

In [ ]:
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()

In [ ]:
image2.size()

In [ ]:
image2